In [ ]:
%load_ext autoreload
%autoreload 2

In [3]:
from droughts_modelling.data import DataFunctions
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import models, layers
from sklearn.preprocessing import RobustScaler
import os
import pandas as pd

1. Sort out data file
2. Check deep learning trainer 2 works - run the evaluate method as this will also train the model
3. Hook the trainings up to GCP and iterate over to get the best model

In [4]:
data_class = DataFunctions()

In [56]:
train_data = data_class.light_weekly_aggregate_train()
val_data = data_class.light_weekly_aggregate_validate()
test_data = data_class.light_weekly_aggregate_test()

In [57]:
test_data

,fips_,year_,week_num_,PRECTOT_mean,PS_mean,QV2M_mean,T2M_mean,T2MDEW_mean,T2MWET_mean,T2M_MAX_mean,...,TS_mean,WS10M_mean,WS10M_MAX_mean,WS10M_MIN_mean,WS10M_RANGE_mean,WS50M_mean,WS50M_MAX_mean,WS50M_MIN_mean,WS50M_RANGE_mean,score_max
0,1001,2019,2,1.604286,100.710000,5.791429,7.820000,5.324286,5.141429,13.990000,...,7.744286,2.435714,3.325714,1.715714,1.610000,4.891429,6.487143,3.340000,3.147143,0.0
1,1001,2019,3,4.114286,100.555714,5.581429,6.798571,4.468571,4.340000,11.945714,...,6.407143,2.378571,3.431429,1.347143,2.087143,4.317143,6.152857,2.728571,3.425714,0.0
2,1001,2019,4,7.530000,100.705714,4.538571,4.972857,1.648571,1.458571,11.154286,...,4.422857,2.618571,3.755714,1.561429,2.192857,4.744286,7.060000,2.927143,4.135714,0.0
3,1001,2019,5,0.878571,100.631429,5.165714,6.250000,3.421429,3.234286,13.391429,...,6.021429,1.984286,2.894286,0.992857,1.898571,3.995714,6.041429,1.671429,4.368571,0.0
4,1001,2019,6,0.574286,100.781429,8.638571,13.088571,10.671429,10.484286,18.827143,...,13.062857,2.570000,3.352857,1.805714,1.545714,5.032857,6.378571,3.410000,2.965714,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
326334,56043,2020,49,0.198571,83.754286,1.977143,-2.270000,-11.144286,-6.707143,3.998571,...,-4.080000,2.752857,3.825714,1.730000,2.098571,4.190000,5.825714,2.747143,3.077143,4.0
326335,56043,2020,50,0.538571,83.165714,2.172857,-2.527143,-10.194286,-6.361429,2.264286,...,-4.081429,2.917143,4.511429,1.295714,3.215714,4.381429,6.402857,2.027143,4.378571,4.0
326336,56043,2020,51,1.041429,82.821429,2.427143,-3.722857,-8.595714,-6.158571,1.544286,...,-5.297143,3.715714,5.474286,2.230000,3.242857,5.808571,8.092857,3.615714,4.475714,4.0
326337,56043,2020,52,0.990000,82.907143,2.564286,-2.982857,-8.382857,-5.681429,2.851429,...,-4.315714,5.145714,7.682857,3.452857,4.227143,7.677143,10.792857,5.254286,5.537143,4.0


In [20]:
df = test_data

In [21]:
ohe = OneHotEncoder(sparse = False)
ohe.fit(df[['score_max']])
scoremax_encoded = ohe.transform(df[['score_max']])
df["score_max_0"],df["score_max_1"],df['score_max_2'],df['score_max_3'],df['score_max_4'],df['score_max_5'] = scoremax_encoded.T 
scaled_test_data_ohe = df.drop(columns=['score_max']) 

In [67]:
scoremax_encoded.T.shape

(6, 326339)

In [63]:
class TestDataPreprocessing():
    
    def __init__(self):
        self.test_data = test_data.copy()
        self.features = self.test_data.drop(columns=['fips_','year_','week_num_','score_max']).columns
        
    def test_robust(self):
        df = self.test_data.copy()
        for f in self.features:
            median = np.median(df[f])
            iqr = np.subtract(*np.percentile(df[f], [75, 25]))
            df[f] = df[f].map(lambda x: (x-median)/iqr)
        
        self.test_scaled_data = df
    
    def test_ohe(self):
        self.test_robust()
        df = self.test_scaled_data.copy()
        ohe = OneHotEncoder(sparse = False)
        ohe.fit(df[['score_max']])
        scoremax_encoded = ohe.transform(df[['score_max']])
        df["score_max_0"],df["score_max_1"],df['score_max_2'],df['score_max_3'],df['score_max_4'],df['score_max_5'] = scoremax_encoded.T 
        self.scaled_test_data_ohe = df.drop(columns=['score_max']) 
        
    
    def test_window(self):
        self.test_ohe()
        self.test_windowed_data = WindowGenerator(self.scaled_test_data_ohe,input_width=6,label_width=6,shift=1,label_columns=["score_max_0","score_max_1","score_max_2","score_max_3","score_max_4","score_max_5"]).make_dataset()
        return self.test_windowed_data

In [64]:
c = TestDataPreprocessing()
scaled = c.test_window()
scaled

<MapDataset shapes: ((None, 6, 27), (None, 6, 6)), types: (tf.float32, tf.float32)>

## Tensorflow window class method

In [6]:
class WindowGenerator():
    def __init__(self,data,input_width,label_width,shift,label_columns=None):

        self.data = data
        
        self.label_columns = label_columns
        if label_columns is not None:
            self.label_columns_indices = {name: i for i, name in enumerate(label_columns)}
            self.column_indices = {name: i for i, name in enumerate(self.data.columns)}

    
        self.input_width = input_width
        self.label_width = label_width
        self.shift = shift

        self.total_window_size = input_width + shift

        self.input_slice = slice(0, input_width)
        self.input_indices = np.arange(self.total_window_size)[self.input_slice]

        self.label_start = self.total_window_size - self.label_width
        self.labels_slice = slice(self.label_start, None)
        self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

    def __repr__(self):
        return '\n'.join([
            f'Total window size: {self.total_window_size}',
            f'Input indices: {self.input_indices}',
            f'Label indices: {self.label_indices}',
            f'Label column name(s): {self.label_columns}'])
    
    
    def split_window(self, list_of_consecutive_inputs_w_labels):
        inputs = list_of_consecutive_inputs_w_labels[:, self.input_slice, :]
        labels = list_of_consecutive_inputs_w_labels[:, self.labels_slice, :]
        if self.label_columns is not None:
            labels = tf.stack([labels[:, :, self.column_indices[name]] for name in self.label_columns],axis=-1)

        inputs.set_shape([None, self.input_width, None])
        labels.set_shape([None, self.label_width, None])

        return inputs, labels

    def make_dataset(self):
        data = np.array(self.data, dtype=np.float32)
        ds = tf.keras.preprocessing.timeseries_dataset_from_array(data=data,targets=None,
                                                              sequence_length=self.total_window_size,
          sequence_stride=1,
          shuffle=True,
          batch_size=32,)

        ds = ds.map(self.split_window)

        return ds

In [68]:
df = train_data

In [69]:
ohe = OneHotEncoder(sparse = False)
ohe.fit(df[['score_max']])
scoremax_encoded = ohe.transform(df[['score_max']])
df["score_max_0"],df["score_max_1"],df['score_max_2'],df['score_max_3'],df['score_max_4'],df['score_max_5'] = scoremax_encoded.T 
df_ = df.drop(columns=['score_max'])
df_

,fips_,year_,week_num_,PRECTOT_mean,PS_mean,QV2M_mean,T2M_mean,T2MDEW_mean,T2MWET_mean,T2M_MAX_mean,...,WS50M_mean,WS50M_MAX_mean,WS50M_MIN_mean,WS50M_RANGE_mean,score_max_0,score_max_1,score_max_2,score_max_3,score_max_4,score_max_5
0,1001,2000,1,5.261429,100.702857,6.491429,9.878571,5.648571,5.674286,16.345714,...,5.044286,6.934286,2.815714,4.120000,0.0,1.0,0.0,0.0,0.0,0.0
1,1001,2000,2,3.554286,101.100000,6.104286,9.662857,5.527143,5.552857,16.751429,...,5.142857,7.162857,2.968571,4.192857,0.0,0.0,1.0,0.0,0.0,0.0
2,1001,2000,3,4.017143,100.348571,6.090000,8.270000,4.834286,4.890000,13.738571,...,4.702857,6.682857,2.997143,3.685714,0.0,0.0,1.0,0.0,0.0,0.0
3,1001,2000,4,3.742857,100.755714,3.072857,0.737143,-3.367143,-3.265714,5.570000,...,4.928571,6.727143,3.244286,3.480000,0.0,0.0,1.0,0.0,0.0,0.0
4,1001,2000,5,0.000000,101.012857,3.142857,2.501429,-2.685714,-2.627143,10.460000,...,4.125714,5.971429,2.045714,3.927143,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2759898,56043,2016,48,0.211429,82.517143,2.265714,-3.541429,-9.130000,-9.007143,0.507143,...,6.231429,8.827143,3.687143,5.140000,1.0,0.0,0.0,0.0,0.0,0.0
2759899,56043,2016,49,0.624286,82.787143,1.841429,-8.072857,-12.847143,-12.498571,-3.115714,...,4.784286,7.161429,2.328571,4.831429,1.0,0.0,0.0,0.0,0.0,0.0
2759900,56043,2016,50,0.754286,82.602857,1.574286,-10.298571,-14.704286,-14.268571,-4.091429,...,4.600000,7.524286,1.565714,5.957143,1.0,0.0,0.0,0.0,0.0,0.0
2759901,56043,2016,51,0.775714,82.742857,2.201429,-5.717143,-9.964286,-9.821429,0.598571,...,6.905714,9.538571,4.108571,5.428571,1.0,0.0,0.0,0.0,0.0,0.0


In [70]:
len(df_.columns)

27

In [71]:
scaling_pipe = Pipeline([('robust_scaler',RobustScaler())])
encoder = Pipeline([('ohe',OneHotEncoder(sparse=False))])
preprocessor = ColumnTransformer([('scaling',scaling_pipe,train_data.drop(columns=['fips_','year_','week_num_','score_max']).columns),
                                  ('cat_encoder',encoder,['score_max'])],remainder='passthrough')
preprocessed_data = preprocessor.fit_transform(train_data)

In [88]:
d = pd.DataFrame(preprocessed_data)

In [94]:
train_data

,fips_,year_,week_num_,PRECTOT_mean,PS_mean,QV2M_mean,T2M_mean,T2MDEW_mean,T2MWET_mean,T2M_MAX_mean,...,WS50M_MAX_mean,WS50M_MIN_mean,WS50M_RANGE_mean,score_max,score_max_0,score_max_1,score_max_2,score_max_3,score_max_4,score_max_5
0,1001,2000,1,5.261429,100.702857,6.491429,9.878571,5.648571,5.674286,16.345714,...,6.934286,2.815714,4.120000,1.0,0.0,1.0,0.0,0.0,0.0,0.0
1,1001,2000,2,3.554286,101.100000,6.104286,9.662857,5.527143,5.552857,16.751429,...,7.162857,2.968571,4.192857,2.0,0.0,0.0,1.0,0.0,0.0,0.0
2,1001,2000,3,4.017143,100.348571,6.090000,8.270000,4.834286,4.890000,13.738571,...,6.682857,2.997143,3.685714,2.0,0.0,0.0,1.0,0.0,0.0,0.0
3,1001,2000,4,3.742857,100.755714,3.072857,0.737143,-3.367143,-3.265714,5.570000,...,6.727143,3.244286,3.480000,2.0,0.0,0.0,1.0,0.0,0.0,0.0
4,1001,2000,5,0.000000,101.012857,3.142857,2.501429,-2.685714,-2.627143,10.460000,...,5.971429,2.045714,3.927143,1.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2759898,56043,2016,48,0.211429,82.517143,2.265714,-3.541429,-9.130000,-9.007143,0.507143,...,8.827143,3.687143,5.140000,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2759899,56043,2016,49,0.624286,82.787143,1.841429,-8.072857,-12.847143,-12.498571,-3.115714,...,7.161429,2.328571,4.831429,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2759900,56043,2016,50,0.754286,82.602857,1.574286,-10.298571,-14.704286,-14.268571,-4.091429,...,7.524286,1.565714,5.957143,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2759901,56043,2016,51,0.775714,82.742857,2.201429,-5.717143,-9.964286,-9.821429,0.598571,...,9.538571,4.108571,5.428571,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [96]:
df = train_data.copy()
ohe = OneHotEncoder(sparse = False)
ohe.fit(df[['score_max']])
scoremax_encoded = ohe.transform(df[['score_max']])
df["score_max_0"],df["score_max_1"],df['score_max_2'],df['score_max_3'],df['score_max_4'],df['score_max_5'] = scoremax_encoded.T 
ohed = df.drop(columns=['score_max'])
ohed

,fips_,year_,week_num_,PRECTOT_mean,PS_mean,QV2M_mean,T2M_mean,T2MDEW_mean,T2MWET_mean,T2M_MAX_mean,...,WS50M_mean,WS50M_MAX_mean,WS50M_MIN_mean,WS50M_RANGE_mean,score_max_0,score_max_1,score_max_2,score_max_3,score_max_4,score_max_5
0,1001,2000,1,5.261429,100.702857,6.491429,9.878571,5.648571,5.674286,16.345714,...,5.044286,6.934286,2.815714,4.120000,0.0,1.0,0.0,0.0,0.0,0.0
1,1001,2000,2,3.554286,101.100000,6.104286,9.662857,5.527143,5.552857,16.751429,...,5.142857,7.162857,2.968571,4.192857,0.0,0.0,1.0,0.0,0.0,0.0
2,1001,2000,3,4.017143,100.348571,6.090000,8.270000,4.834286,4.890000,13.738571,...,4.702857,6.682857,2.997143,3.685714,0.0,0.0,1.0,0.0,0.0,0.0
3,1001,2000,4,3.742857,100.755714,3.072857,0.737143,-3.367143,-3.265714,5.570000,...,4.928571,6.727143,3.244286,3.480000,0.0,0.0,1.0,0.0,0.0,0.0
4,1001,2000,5,0.000000,101.012857,3.142857,2.501429,-2.685714,-2.627143,10.460000,...,4.125714,5.971429,2.045714,3.927143,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2759898,56043,2016,48,0.211429,82.517143,2.265714,-3.541429,-9.130000,-9.007143,0.507143,...,6.231429,8.827143,3.687143,5.140000,1.0,0.0,0.0,0.0,0.0,0.0
2759899,56043,2016,49,0.624286,82.787143,1.841429,-8.072857,-12.847143,-12.498571,-3.115714,...,4.784286,7.161429,2.328571,4.831429,1.0,0.0,0.0,0.0,0.0,0.0
2759900,56043,2016,50,0.754286,82.602857,1.574286,-10.298571,-14.704286,-14.268571,-4.091429,...,4.600000,7.524286,1.565714,5.957143,1.0,0.0,0.0,0.0,0.0,0.0
2759901,56043,2016,51,0.775714,82.742857,2.201429,-5.717143,-9.964286,-9.821429,0.598571,...,6.905714,9.538571,4.108571,5.428571,1.0,0.0,0.0,0.0,0.0,0.0


In [93]:
d.loc[::,15:]

,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32
0,-0.194828,-0.105528,-0.155185,-1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1001.0,2000.0,1.0
1,-0.125862,-0.015913,-0.118202,-1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1001.0,2000.0,2.0
2,-0.270690,0.000838,-0.375635,-1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1001.0,2000.0,3.0
3,-0.257328,0.145729,-0.480058,-1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1001.0,2000.0,4.0
4,-0.485345,-0.556951,-0.253082,-1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1001.0,2000.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2756791,0.376293,0.405360,0.362582,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,56043.0,2016.0,48.0
2756792,-0.126293,-0.391122,0.205946,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,56043.0,2016.0,49.0
2756793,-0.016810,-0.838358,0.777375,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,56043.0,2016.0,50.0
2756794,0.590948,0.652429,0.509065,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,56043.0,2016.0,51.0


In [85]:
pd.DataFrame(preprocessed_data)[30].unique()

array([ 1001.,  1003.,  1005., ..., 56039., 56041., 56043.])

In [15]:
pd.DataFrame(preprocessed_data,columns=df.columns)
#windowed_data = WindowGenerator(preprocessed_data,input_width=6,label_width=6,shift=1,
        #label_columns=["score_max_0","score_max_1","score_max_2","score_max_3","score_max_4","score_max_5"]).make_dataset()

ValueError: Shape of passed values is (2756796, 33), indices imply (2756796, 28)

In [ ]:
class DeepLearning:
    
    def __init__(self,data):
        self.data = data
        #self.data = DataFunctions().light_weekly_aggregate()
        self.features = self.data.drop(columns=['fips_','year','week_num_','score_max']).columns
        
    def robust(self):
        df = self.data.copy()
        for f in self.features:
            median = np.median(df[f])
            IQR = np.subtract(*np.percentile(df[f], [75, 25]))
            df[f] = df[f].map(lambda x: (x-median)/IQR)
        
        self.scaled_data = df
            
    def preprocess(self):
        self.robust()
        self.preprocessed_data = WindowGenerator(self.scaled_data,input_width=6, label_width=1, shift=1,label_columns=['score_max']).make_dataset()
        
    def initialize_model(self):
        self.model = models.Sequential()
        self.model.add(layers.LSTM(20))
        self.model.add(layers.Dense(1,activation='sigmoid'))
        self.model.compile(loss='binary_crossentropy',optimizer='rmsprop',metrics=['accuracy'])
    
    def train_model(self):
        self.initialize_model()
        self.preprocess()
        self.model.fit(self.preprocessed_data,epochs=1,batch_size=32,verbose=1)

In [ ]:
scaled_train_data = robust(train_data)[:int((len(train_data)*0.1))]
scaled_val_data = robust(val_data)[:int((len(train_data)*0.1))]

In [ ]:
#ohe train
from sklearn.preprocessing import OneHotEncoder
train_ohe = OneHotEncoder(sparse = False)
train_ohe.fit(scaled_train_data[['score_max']])
scoremax_encoded = train_ohe.transform(scaled_train_data[['score_max']])
scaled_train_data["score_max_0"],scaled_train_data["score_max_1"],scaled_train_data['score_max_2'],scaled_train_data['score_max_3'],scaled_train_data['score_max_4'],scaled_train_data['score_max_5'] = scoremax_encoded.T 
scaled_train_data_ohe = scaled_train_data.drop(columns=['score_max'])

#ohe val
val_ohe = OneHotEncoder(sparse = False)
val_ohe.fit(scaled_val_data[['score_max']])
scoremax_encoded = val_ohe.transform(scaled_val_data[['score_max']])
scaled_val_data["score_max_0"],scaled_val_data["score_max_1"],scaled_val_data['score_max_2'],scaled_val_data['score_max_3'],scaled_val_data['score_max_4'],scaled_val_data['score_max_5'] = scoremax_encoded.T 
scaled_val_data_ohe = scaled_val_data.drop(columns=['score_max'])



In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

def initialize_model():
    model = models.Sequential()
    model.add(layers.LSTM(32,return_sequences=True,activation='tanh'))
    model.add(layers.LSTM(32,return_sequences=True,activation='tanh'))
    model.add(layers.Dense(20,activation='relu'))
    model.add(layers.Dense(6,activation='softmax'))
    model.compile(loss='categorical_crossentropy',optimizer='rmsprop',metrics=['accuracy'])
    return model
    
def train_model(train_data,validation_data):
    model = initialize_model()
    history = model.fit(train_data,validation_data=validation_data,epochs=1000,batch_size=32,callbacks=EarlyStopping(patience=10,restore_best_weights=True),verbose=1)
    return history

In [ ]:
preprocessed_scaled_ohe = WindowGenerator(scaled_train_data_ohe,input_width=6,label_width=6,shift=1,label_columns=["score_max_0","score_max_1","score_max_2","score_max_3","score_max_4","score_max_5"]).make_dataset()
preprocessed_scaled_ohe.cache

In [ ]:
hist = train_model(preprocessed_train_ohe,preprocessed_val_ohe)

In [ ]:
#predicted_classes = model.predict(X_test > 0.5).astype("int32")[:, 0]
#print(f'test recall score = {recall_score(y_test,predicted_classes)}')
fig, ax = plt.subplots(1,2,figsize=(13,5))
ax[0].plot(hist.history['loss'],label='train_loss')
ax[0].plot(hist.history['val_loss'],label='val_loss')
ax[0].legend()
ax[1].plot(hist.history['accuracy'],label='train_accuracy')
ax[1].plot(hist.history['val_accuracy'],label='val_accuracy')
ax[1].legend()

In [9]:
#Pipeline

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn import set_config; set_config(display='diagram')

class Window(): 
    
    def fit(self,data):
        self.data = data
        self.window = WindowGenerator(self.data,input_width=6,label_width=6,shift=1,label_columns=['score_max_0','score_max_1','score_max_2','score_max_3','score_max_4','score_max_5'])
        return self
 
    def transform(self,data):
        data_windowed = self.window.make_dataset()
        return data_windowed

def initialize_model():
    model = models.Sequential()
    model.add(layers.LSTM(32,return_sequences=True,activation='tanh'))
    model.add(layers.LSTM(32,return_sequences=True,activation='tanh'))
    model.add(layers.Dense(20,activation='relu'))
    model.add(layers.Dense(6,activation='softmax'))
    model.compile(loss='categorical_crossentropy',optimizer='rmsprop',metrics=['accuracy'])
    return model

scaling_pipe = Pipeline([('robust_scaler',RobustScaler())])
encoder = Pipeline([('ohe',OneHotEncoder())])
preprocessor = ColumnTransformer([('scaling',scaling_pipe,train_data.drop(columns=['fips_','year_','week_num_','score_max']).columns),('cat_encoder',encoder,['score_max'])],remainder='passthrough')

window_generator = Pipeline([('window',Window())])
windower = ColumnTransformer([('window',window_generator,[col for col in scaled_train_data_ohe.columns])])

keras_wrapper_model = KerasClassifier(build_fn=initialize_model,epochs=1,batch_size=32,callbacks=EarlyStopping(patience=10,restore_best_weights=True),verbose=1)

final_pipe = Pipeline([('preprocessing', preprocessor),('window_generator',windower),('KerasClassifier', keras_wrapper_model)])
final_pipe_trained = final_pipe.fit(train_data)
final_pipe

NameError: name 'scaled_train_data_ohe' is not defined

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn import set_config; set_config(display='diagram')

from tensorflow.keras import models,layers
from sklearn.preprocessing import OneHotEncoder
from droughts_modelling.data import DataFunctions
from droughts_modelling.window_gen import WindowGenerator
import numpy as np

class DeepLearning2():
    
    def __init__(self):
        self.train_data = DataFunctions().light_weekly_aggregate_train()
        self.test_data = DataFunctions().light_weekly_aggregate_test()
        self.features = self.data.drop(columns=['fips_','year_','week_num_','score_max']).columns
    
    #Train data preprocessing
    def train_robust(self):
        df = self.train_data.copy()
        for f in self.features:
            self.train_median = np.median(df[f])
            self.train_iqr = np.subtract(*np.percentile(df[f], [75, 25]))
            df[f] = df[f].map(lambda x: (x-self.train_median)/self.train_iqr)
            
        self.train_scaled_data = df
        
    def train_ohe(self):
        self.train_robust()
        df = self.train_scaled_data.copy()
        ohe = OneHotEncoder(sparse = False)
        ohe.fit(df[['score_max']])
        scoremax_encoded = ohe.transform(df[['score_max']])
        df["score_max_0"],df["score_max_1"],df['score_max_2'],df['score_max_3'],df['score_max_4'],df['score_max_5'] = scoremax_encoded.T 
        self.scaled_train_data_ohe = df.drop(columns=['score_max'])
        
    def train_window(self):
        self.train_ohe()
        self.train_windowed_data = WindowGenerator(self.scaled_train_data_ohe,input_width=6,label_width=6,shift=1,label_columns=["score_max_0","score_max_1","score_max_2","score_max_3","score_max_4","score_max_5"]).make_dataset()
    
    #Test data preprocessing
    def test_robust(self):
        df = self.test_data.copy()
        for f in self.features:
            self.train_median = np.median(df[f])
            self.train_iqr = np.subtract(*np.percentile(df[f], [75, 25]))
            df[f] = df[f].map(lambda x: (x-self.train_median)/self.train_iqr)
        
        self.test_scaled_data = df
    
    def test_ohe(self):
        self.test_robust()
        df = self.test_scaled_data.copy()
        ohe = OneHotEncoder(sparse = False)
        ohe.fit(df[['score_max']])
        scoremax_encoded = ohe.transform(df[['score_max']])
        df["score_max_0"],df["score_max_1"],df['score_max_2'],df['score_max_3'],df['score_max_4'],df['score_max_5'] = scoremax_encoded.T 
        self.scaled_test_data_ohe = df.drop(columns=['score_max']) 
    
    def test_window(self):
        self.test_ohe()
        self.test_windowed_data = WindowGenerator(self.scaled_test_data_ohe,input_width=6,label_width=6,shift=1,label_columns=["score_max_0","score_max_1","score_max_2","score_max_3","score_max_4","score_max_5"]).make_dataset()
    
    #Model + evaluation
    def initialize_model(self):
        self.model = models.Sequential()
        self.model.add(layers.LSTM(32,return_sequences=True,activation='tanh'))
        self.model.add(layers.LSTM(32,return_sequences=True,activation='tanh'))
        self.model.add(layers.Dense(20,activation='relu'))
        self.model.add(layers.Dense(6,activation='softmax'))
        self.model.compile(loss='categorical_crossentropy',optimizer='rmsprop',metrics=['accuracy'])
        
    def train_model(self):
        self.initialize_model()
        self.train_window()
        self.model.fit(self.train_windowed_data,epochs=1000,batch_size=32,verbose=1)
        
    def evaluate_model(self):
        self.train_model()
        self.test_window()
        self.model.evauluate(self.test_windowed_data,verbose=1)
    

### Manual coding method

In [ ]:
def split_subsample_sequence(df, length):
    df_sample = df.sample(length)
    X_sample = df_sample.iloc[:-1].copy()
    y_sample = float(df_sample[['score_max']].iloc[-1])
    
    return X_sample, y_sample

def get_X_y(df, n_sequences, length):
    X = [split_subsample_sequence(df, length)[0] for n in range(n_sequences)]
    y = [split_subsample_sequence(df, length)[1] for n in range(n_sequences)]
    
    
    return np.array(X),np.array(y)

In [ ]:
d = data[:1000]

In [ ]:
X,y = get_X_y(d, 2000, 21)

In [ ]:
def split_subsample_sequence(df, length):
    df_sample = df.sample(length,random_state=1)
    X_sample = df_sample.iloc[:-1].copy()
    y_sample = float(df_sample[['score_max']].iloc[-1])
    
    return X_sample, y_sample

def get_X_y(df, n_sequences,length):
    X = []
    y = []
    for n in range(n_sequences):
        sequence = split_subsample_sequence(df, length)
        X.append(sequence[0])
        y.append(sequence[1])
    
    return np.array(X),np.array(y)

def get_X_y_by_geolocation(df, n_sequences, length):
    X = []
    y = []
    for fip in sorted(set(df['fips_'])):
        fip_df = df[df['fips_'] == fip]
        sequences = get_X_y(fip_df,n_sequences,length)
        X.append(sequences[0])
        y.append(sequences[1])
        

    return np.array(X),np.array(y)